In [1]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0


In [20]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"

lat_long_list = [
    [-6.219151, 106.801624],
    [-6.219972, 106.802643],
    [-6.219679, 106.803190],
    [-6.219167, 106.803645],
    [-6.218602, 106.803914],
    [-6.218090, 106.803758],
    [-6.217338, 106.803271],
    [-6.216970, 106.802671],
    [-6.217263, 106.802112],
    [-6.217887, 106.801549],
    [-6.218538, 106.801346]
]

# Define column names
columns = ['date', 'temperature_2m', 'wind_speed_10m', 'wind_direction_10m']

# Store rows in a list
all_data = [] 

for lat_long in lat_long_list:
    params = {
        "latitude": lat_long[0],
        "longitude": lat_long[1],
        "hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m"],
        "timezone": "Asia/Bangkok",
        "start_date": "2025-06-11",
        "end_date": "2025-06-11"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Latitude {lat_long[0]} and Longitude {lat_long[1]}")
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
#     print(f"Elevation {response.Elevation()} m asl")
#     print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
#     print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["latitude"] = lat_long[0]
    hourly_data["longitude"] = lat_long[1]

    hourly_df = pd.DataFrame(hourly_data)
    hourly_df["date"] = hourly_df["date"].dt.tz_convert("Asia/Jakarta")
    hourly_df_filtered = hourly_df[hourly_df["date"].dt.hour.isin([20, 21, 22])]

    all_data.append(hourly_df_filtered)  # Store filtered results

final_df = pd.concat(all_data, ignore_index=True)

Latitude -6.219151 and Longitude 106.801624
Coordinates -6.25°N 106.75°E
Latitude -6.219972 and Longitude 106.802643
Coordinates -6.25°N 106.75°E
Latitude -6.219679 and Longitude 106.80319
Coordinates -6.25°N 106.75°E
Latitude -6.219167 and Longitude 106.803645
Coordinates -6.25°N 106.75°E
Latitude -6.218602 and Longitude 106.803914
Coordinates -6.25°N 106.75°E
Latitude -6.21809 and Longitude 106.803758
Coordinates -6.25°N 106.75°E
Latitude -6.217338 and Longitude 106.803271
Coordinates -6.25°N 106.75°E
Latitude -6.21697 and Longitude 106.802671
Coordinates -6.25°N 106.75°E
Latitude -6.217263 and Longitude 106.802112
Coordinates -6.25°N 106.75°E
Latitude -6.217887 and Longitude 106.801549
Coordinates -6.25°N 106.75°E
Latitude -6.218538 and Longitude 106.801346
Coordinates -6.25°N 106.75°E


In [21]:
final_df

,date,temperature_2m,wind_speed_10m,wind_direction_10m,latitude,longitude
0,2025-06-11 20:00:00+07:00,27.910500,5.506941,101.309898,-6.219151,106.801624
1,2025-06-11 21:00:00+07:00,27.360500,3.415260,108.435043,-6.219151,106.801624
2,2025-06-11 22:00:00+07:00,27.110500,1.938659,111.801476,-6.219151,106.801624
3,2025-06-11 20:00:00+07:00,27.722000,5.506941,101.309898,-6.219972,106.802643
4,2025-06-11 21:00:00+07:00,27.172001,3.415260,108.435043,-6.219972,106.802643
5,2025-06-11 22:00:00+07:00,26.922001,1.938659,111.801476,-6.219972,106.802643
6,2025-06-11 20:00:00+07:00,27.722000,5.506941,101.309898,-6.219679,106.803190
7,2025-06-11 21:00:00+07:00,27.172001,3.415260,108.435043,-6.219679,106.803190
8,2025-06-11 22:00:00+07:00,26.922001,1.938659,111.801476,-6.219679,106.803190
9,2025-06-11 20:00:00+07:00,27.734999,5.506941,101.309898,-6.219167,106.803645
